<a href="https://colab.research.google.com/github/aviyashar/application-prod/blob/main/%D7%A4%D7%99%D7%A7%D7%95%D7%93_%D7%94%D7%A2%D7%95%D7%A8%D7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 6.5 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=b1e5e41022fe54867c52206907b4325310540d89f1fd50bf520f0deb24422bfb
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [2]:
import asyncio
from telethon.sync import TelegramClient
from telethon.sessions import StringSession

api_id = 21722549
api_hash = '''e55242de46571ce209358765db0bfe25'''
string_session = '1BJWap1sBu44s6_hQHku-mCEWwRnrNAuRzbeQQJ6QWQUVI4WcIiOvxrk1b9gTvULrDCX-iN0f1nwUV0VcSH1qbpz6FdHKLGYNf-LV0YRNp7FtHxbA6Q0UMFCET8J3hbCQbT-uzhN5UoclPa-yCzZLRSE1NvFTwetzcc9wGcbzMuMiVh1YDuxEjbN50SbF7roASGwbzTOhoUnxS2Ew9Tww96M_HRWcKKgABaHp-DU-59p86b06SPqRs_QypdLHjyvQJ2K8aDhiAQG6DvGiOXe128AAl-KrdA83IyQ0pp3NPOH3it0AfPs5R02g5nZjkXe24OpeKNyTAFm0rP7GIkiFcG_2Wzc2VgU='
channel_username = 'PikudHaOref_all'
import pandas as pd
import re
from datetime import datetime




async def main():
    async with TelegramClient(StringSession(string_session), api_id, api_hash) as client:
        # Fetch the channel entity using its username
        channel = await client.get_entity(channel_username)  # Use await here

        # Get the message history
        messages = []
        async for message in client.iter_messages(channel, limit=100000):  # Use async for
            messages.append(message.text)
        return messages  # Return the messages if needed

# Instead of asyncio.run(main()), use the following in a Jupyter Notebook:
x = await main()

In [3]:
# יצירת DataFrame
df = pd.DataFrame(x, columns=["message"])

# סינון החוצה של הודעות אם מתקיים לפחות אחד מהתנאים:
# - ההודעה כוללת "מבזק"
# - ההודעה כוללת "האירוע הסתיים"
# - ההודעה לא כוללת תאריך

def should_exclude(msg):
    if not isinstance(msg, str):
        return True
    if 'מבזק' in msg:
        return True
    if 'האירוע הסתיים' in msg:
        return True
    if 'עדכון' in msg:
        return True
    return False

df = df[~df['message'].apply(should_exclude)]

# שליפה של event מתוך השורה הראשונה של ההודעה
def extract_event(msg):
    first_line = msg.split('\n')[0]
    clean = re.sub(r'[\*🚨✈]', '', first_line).strip()
    return clean.split('(')[0].strip()

df['event'] = df['message'].apply(extract_event)

# שליפת datetime מתוך השורה הראשונה
pattern = r'(\d{1,2}/\d{1,2}/\d{4})\)?\s+(\d{1,2}:\d{2})'
def extract_datetime(text):
    match = re.search(pattern, text)
    if match:
        date, time = match.groups()
        return datetime.strptime(f"{date} {time}", "%d/%m/%Y %H:%M")
    return None

df['datetime'] = df['message'].apply(extract_datetime)

# שליפת אזור ויישובים

def process_message(message):
    lines = message.split('\n')
    current_area = None
    processed_data = []

    for line in lines:
        if '**אזור' in line:
            current_area = re.sub(r'[\*]', '', line).strip().replace('אזור', '').strip()
        elif current_area and line.strip() and not any(kw in line for kw in ['היכנסו', 'האירוע הסתיים', 'הודעה חדשה', 'על תושבי']):
            city_block = line.split('(')[0].strip() if '(' in line else line.strip()
            city_list = [c.strip() for c in city_block.split(',') if c.strip()]
            for city in city_list:
                processed_data.append({'area': current_area, 'city': city})

    return processed_data if processed_data else None

# החלת הפונקציה
df['processed_data'] = df['message'].apply(process_message)
df = df.explode('processed_data')

# חילוץ שדות area ו-city
df['area'] = df['processed_data'].apply(lambda x: x['area'] if x else None)
df['city'] = df['processed_data'].apply(lambda x: x['city'] if x else None)

# ניקוי וסידור סופי
df = df.reset_index(drop=True)

# # סינון לפי תאריך מסוים
filter_date = datetime(2023, 10, 7)
df = df[df['datetime'] >= filter_date]


In [4]:
df

,message,event,datetime,processed_data,area,city
0,🚨 **ירי רקטות וטילים (22/6/2025) 8:03**\n\n**א...,ירי רקטות וטילים,2025-06-22 08:03:00,"{'area': 'גליל תחתון', 'city': 'כפר כמא'}",גליל תחתון,כפר כמא
1,🚨 **ירי רקטות וטילים (22/6/2025) 8:03**\n\n**א...,ירי רקטות וטילים,2025-06-22 08:03:00,"{'area': 'גליל תחתון', 'city': 'שרונה'}",גליל תחתון,שרונה
2,🚨 **ירי רקטות וטילים (22/6/2025) 8:03**\n\n**א...,ירי רקטות וטילים,2025-06-22 08:03:00,"{'area': 'גולן דרום', 'city': 'מבוא חמה'}",גולן דרום,מבוא חמה
3,🚨 **ירי רקטות וטילים (22/6/2025) 8:03**\n\n**א...,ירי רקטות וטילים,2025-06-22 08:03:00,"{'area': 'גולן דרום', 'city': 'האון'}",גולן דרום,האון
4,🚨 **ירי רקטות וטילים (22/6/2025) 8:03**\n\n**א...,ירי רקטות וטילים,2025-06-22 08:03:00,"{'area': 'גולן דרום', 'city': 'מעגן'}",גולן דרום,מעגן
...,...,...,...,...,...,...
36516,✈ **חדירת כלי טיס עוין (25/2/2025) 8:39**\n\n*...,חדירת כלי טיס עוין,2025-02-25 08:39:00,"{'area': 'גולן דרום', 'city': 'באזורים אלו יש ...",גולן דרום,באזורים אלו יש להיכנס למרחב המוגן ולשהות בו למ...
36517,✈ **חדירת כלי טיס עוין (25/2/2025) 8:39**\n\n*...,חדירת כלי טיס עוין,2025-02-25 08:39:00,"{'area': 'גולן דרום', 'city': 'אלא אם ניתנה הת...",גולן דרום,אלא אם ניתנה התרעה נוספת.
36518,✈ **חדירת כלי טיס עוין (25/2/2025) 8:39**\n\n*...,חדירת כלי טיס עוין,2025-02-25 08:39:00,"{'area': 'גולן דרום', 'city': 'להנחיות המלאות ...",גולן דרום,להנחיות המלאות - https://www.oref.org.il/heb/l...
36519,🚨 **ירי רקטות וטילים (8/2/2025) 15:53**\n\n**א...,ירי רקטות וטילים,2025-02-08 15:53:00,"{'area': 'עוטף עזה', 'city': 'נירים'}",עוטף עזה,נירים


In [5]:
df.to_excel('alerts2.xlsx', index=False)